In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('train.tsv', sep='\t')
# df_train.head()
df_test = pd.read_csv('test.tsv', sep='\t')
df_test

,id,statuses_count,default_profile,default_profile_image,friends_count,followers_count,favourites_count,geo_enabled,listed_count,account_age_hours,diversity,mean_mins_between_tweets,mean_tweet_length,mean_retweets,reply_rate
0,1,1990,0,0,546,1225,133,0,53,21007,0.869048,2208.218750,90.363636,2.500000,0.121212
1,4,601,0,0,948,368,212,0,32,26934,0.612211,292.957143,65.829787,2.000000,0.609929
2,5,17,0,0,301,39,1,0,5,25497,0.879310,116383.545455,98.666667,1.000000,0.083333
3,6,271,0,0,126,62,3,0,2,70240,0.741379,24589.039682,75.204724,3.000000,0.157480
4,8,8,1,0,960,146,24,0,0,12162,0.500000,32.428571,35.750000,1.000000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,3167,1179,1,0,1743,792,336,1,30,39072,0.780886,4768.979798,119.670000,1.689655,0.400000
1584,3168,125,0,0,429,2356,23,0,261,39527,0.735931,17054.462963,106.935780,3.606061,0.587156
1585,3169,148,0,0,861,226,12,0,9,39263,0.636042,12397.927928,81.357143,1.777778,0.589286
1586,3172,1065,0,0,929,263,2866,0,15,58959,0.871287,16775.045454,111.347826,1.333333,0.521739


In [4]:
train_x = df_train.drop(['id','bot'], axis=1)
train_y = df_train['bot']
# train_x.head()
test_x = df_test.drop(['id'], axis=1)
test_x.head()

,statuses_count,default_profile,default_profile_image,friends_count,followers_count,favourites_count,geo_enabled,listed_count,account_age_hours,diversity,mean_mins_between_tweets,mean_tweet_length,mean_retweets,reply_rate
0,1990,0,0,546,1225,133,0,53,21007,0.869048,2208.218750,90.363636,2.5,0.121212
1,601,0,0,948,368,212,0,32,26934,0.612211,292.957143,65.829787,2.0,0.609929
2,17,0,0,301,39,1,0,5,25497,0.879310,116383.545455,98.666667,1.0,0.083333
3,271,0,0,126,62,3,0,2,70240,0.741379,24589.039682,75.204724,3.0,0.157480
4,8,1,0,960,146,24,0,0,12162,0.500000,32.428571,35.750000,1.0,0.250000


In [5]:
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Perceptron

kf = KFold(n_splits=10, shuffle=True)

for tr_idx, val_idx in kf.split(train_x):
    tr_x, val_x = train_x.iloc[tr_idx], train_x.iloc[val_idx]
    tr_y, val_y = train_y.iloc[tr_idx], train_y.iloc[val_idx]
    # model = DecisionTreeClassifier()
    model = lgb.LGBMClassifier()
    model.fit(tr_x, tr_y)
    val_pred = model.predict(val_x)
    score = f1_score(val_y, val_pred)
    print(f'f1: {score:.4f}')


[LightGBM] [Info] Number of positive: 221, number of negative: 1208
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000309 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2796
[LightGBM] [Info] Number of data points in the train set: 1429, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.154654 -> initscore=-1.698559
[LightGBM] [Info] Start training from score -1.698559
f1: 0.8293
[LightGBM] [Info] Number of positive: 214, number of negative: 1215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2798
[LightGBM] [Info] Number of data points in the train set: 1429, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.149755 -> initscore=-1.736523

In [6]:
test_pred = model.predict(test_x)


In [7]:
df_submit = pd.DataFrame()
df_submit['id'] = df_test.iloc[:, [0]]
df_submit['result'] = test_pred
df_submit.head()

,id,result
0,1,0
1,4,0
2,5,0
3,6,0
4,8,1


In [8]:
df_submit.to_csv('submit.csv', header=False, index=False)